# Lab22 MNIST with CNN Model

In [2]:
import torch
import torchvision.datasets as dsets
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.init
import random

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [4]:
#parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
device

'cuda'

In [5]:
#MNIST dataset

mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train = False,
                        transform = transforms.ToTensor(),
                        download = True)

0it [00:00, ?it/s]

9920512it [00:01, 5472146.41it/s]                             


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


0it [00:00, ?it/s]

32768it [00:00, 64715.92it/s]            
0it [00:00, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw


1654784it [00:01, 1405685.66it/s]                            
0it [00:00, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw


8192it [00:00, 19597.71it/s]            

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw
Processing...
Done!


In [6]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size = batch_size,
                                         shuffle = True,
                                         drop_last = True)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0),  -1)
        out = self.fc(out)
        return out

In [8]:
model = CNN().to(device)

In [9]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [10]:
# training
total_batch = len(data_loader)

for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

[Epoch:1] cost = 0.22390058636665344
[Epoch:2] cost = 0.06212501972913742
[Epoch:3] cost = 0.04486560449004173
[Epoch:4] cost = 0.03549722954630852
[Epoch:5] cost = 0.028972478583455086
[Epoch:6] cost = 0.024913422763347626
[Epoch:7] cost = 0.02073877863585949
[Epoch:8] cost = 0.0180584117770195
[Epoch:9] cost = 0.015226406045258045
[Epoch:10] cost = 0.012647124007344246
[Epoch:11] cost = 0.010213012807071209
[Epoch:12] cost = 0.009781856089830399
[Epoch:13] cost = 0.008124522864818573
[Epoch:14] cost = 0.0076422179117798805
[Epoch:15] cost = 0.006509162951260805
Learning Finished!


In [11]:
print(device)
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

cuda
Accuracy: 0.9882999658584595


/home/bigcloud/anaconda3/envs/hyunsoo/lib/python3.6/site-packages/torchvision/datasets/mnist.py:58: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/bigcloud/anaconda3/envs/hyunsoo/lib/python3.6/site-packages/torchvision/datasets/mnist.py:48: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
